In [1]:
## Reading from the Watch folder, prior classified into whiebackground

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np
import os
from pathlib import Path
import shutil

# Load a pre-trained model (e.g., ResNet50)
model = tf.keras.applications.ResNet50(weights='imagenet')

In [2]:
## Initial Statistics: original images (image_folder), 
## transparent images (transparent_images), white background (white_bg_images)

In [3]:
print("Total Number of original files")

folder_path = Path("image_folder")
num_items = len(list(folder_path.iterdir()))

print(f"Number of items in '{folder_path}': {num_items}")

Total Number of original files
Number of items in 'image_folder': 1679


In [4]:
folder_path = Path("transparent_images")
num_items = len(list(folder_path.iterdir()))

print(f"Number of items in '{folder_path}': {num_items}")

Number of items in 'transparent_images': 341


In [5]:
folder_path = Path("white_bg_images")
num_items = len(list(folder_path.iterdir()))

print(f"Number of items in '{folder_path}': {num_items}")

Number of items in 'white_bg_images': 1116


In [6]:

def categorize_watch_images_concurrent(data_dir, good_watch_folder, bad_watch_folder):
    # create list to track all watches with white background, of those with watch images, of those with not

    
    """ Process all images in a given folder concurrently """
        
    if os.path.exists(good_watch_folder):
        try:
            shutil.rmtree(good_watch_folder)
            print(f"Directory '{good_watch_folder}' and all its contents have been deleted.")
        except Exception as e:
            print(f"Error: Failed to delete '{good_watch_folder}'. {e}")
            
    if os.path.exists(bad_watch_folder):
        try:
            shutil.rmtree(bad_watch_folder)
            print(f"Directory '{bad_watch_folder}' and all its contents have been deleted.")
        except Exception as e:
            print(f"Error: Failed to delete '{bad_watch_folder}'. {e}")
            
    os.makedirs(good_watch_folder)
    os.makedirs(bad_watch_folder)

    image_files = [f for f in os.listdir(data_dir) ]
    image_paths = [os.path.join(data_dir, filename) for filename in image_files]

    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(watch_classify, image_path, good_watch_folder, bad_watch_folder) for image_path in image_paths]
        with tqdm(total=len(futures), desc="Processing images") as progress_bar:
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    print(f"An error occurred: {e}")
                finally:
                    progress_bar.update(1)
     
 # Document processed watch images in webpages   

    with open("good_watch_classification_files.html", "w") as f:
            f.write("<html><head><style>img {width:100px;height:100px;background-color:#39F;}</style></head><body>")      
            for item in good_watch:
                f.write(f"{item}")
            f.write(f"</body></html>")            


    with open("bad_watch.html", "w") as f:
            f.write("<html><head><style>img {width:100px;height:100px;background-color:#39F;}</style></head><body>")      
            for item in bad_watch:
                f.write(f"{item}")
            f.write(f"</body></html>")   
            
    with open("bad_file.txt", "w") as f:
            for item in bad_file:
                f.write(f"{item}")
                    
# Load and preprocess an image

def watch_classify(image_path, good_watch_folder, bad_watch_folder):
        filename = os.path.basename(image_path)
        try:
            img = image.load_img(image_path) #original image - I will save this in its original size - unchanged from part 1
            check_img = image.load_img(image_path, target_size=(224,224))  #This has to be changed to 224 x 224 to use library
            x = image.img_to_array(check_img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # Make predictions
            preds = model.predict(x)
            # Decode predictions (get the top 5 predicted classes)
            decoded_preds = decode_predictions(preds, top=5)[0]
           
            # Print the top predictions
            for i, (imagenet_id, label, score) in enumerate(decoded_preds):
                print(f"{i + 1}: {label} ({score:.2f})")
                
            watch_flag = 0
            # Check if "watch" is among the top predictions
            for imagenet_id, label, score in decoded_preds:
                if 'watch' in label.lower() or 'clock' in label.lower():
                    print(f"Found 'watch' with probability {score:.2f}")
                    good_watch.append(f"<a href='{filename}'><img src='{image_path}'> {filename} {score:.2f} </a>  ")
                    image_save_path = os.path.join(good_watch_folder, filename)
                    print("image_save_path_good",image_save_path)
                    img.save(image_save_path)  
                    watch_flag = 1
                    break
                    
            if watch_flag == 0:
                    bad_watch.append(f"<a href='{image_path}'><img src='{image_path}'> {filename} {score:.2f} </a>  ")
                    image_save_path = os.path.join(bad_watch_folder, filename)
                    print("image_save_path_bad",image_save_path)
                    img.save(image_save_path)                
                    
        except Exception as e:
            bad_file.append(f"{image_path}\n")
            print(f"Error: {e}")




In [7]:

if __name__ == "__main__":
    img_path = ''
    good_watch =[]
    bad_watch =[]
    bad_file=[]
    data_dir = r"white_bg_images"
    good_watch_folder = r"good_watch_folder"   #combine good white backgroung images with transparency images
    bad_watch_folder = r"bad_watch_folder"
    categorize_watch_images_concurrent(data_dir, good_watch_folder, bad_watch_folder)


Directory 'good_watch_folder' and all its contents have been deleted.
Directory 'bad_watch_folder' and all its contents have been deleted.


Processing images:   0%|          | 0/1116 [00:00<?, ?it/s]

1: stopwatch (0.93)
2: analog_clock (0.06)
3: digital_watch (0.00)
4: wall_clock (0.00)
5: barometer (0.00)
Found 'watch' with probability 0.93
image_save_path_good good_watch_folder\015f369d469109003f4a70f5133b6799.png
1: stopwatch (0.99)
2: analog_clock (0.00)
3: digital_watch (0.00)
4: barometer (0.00)
5: magnetic_compass (0.00)
Found 'watch' with probability 0.99
image_save_path_good good_watch_folder\0265422981bb3f66d540ab66abf592d3.png
1: stopwatch (0.91)
2: analog_clock (0.08)
3: digital_watch (0.01)
4: magnetic_compass (0.00)
5: wall_clock (0.00)
Found 'watch' with probability 0.91
image_save_path_good good_watch_folder\01a08adf2c8218cd60285d26b28b5be2.png
1: stopwatch (0.64)
2: analog_clock (0.33)
3: wall_clock (0.01)
4: digital_watch (0.01)
5: magnetic_compass (0.00)
Found 'watch' with probability 0.64
image_save_path_good good_watch_folder\041b88e328d1dc95fad311d2399e2a17.png
1: stopwatch (0.69)
2: analog_clock (0.19)
3: barometer (0.09)
4: digital_watch (0.01)
5: wall_clock

Processing images:   1%|          | 7/1116 [00:06<10:41,  1.73it/s]  

1: stopwatch (0.98)
2: digital_watch (0.01)
3: analog_clock (0.00)
4: barometer (0.00)
5: reflex_camera (0.00)
Found 'watch' with probability 0.98
image_save_path_good good_watch_folder\0226b246d82fb196b353eee85e0c7b3b.png
1: analog_clock (0.75)
2: stopwatch (0.17)
3: wall_clock (0.06)
4: odometer (0.01)
5: barometer (0.01)
Found 'watch' with probability 0.75
image_save_path_good good_watch_folder\02af40493eeb9c41cbee5b4a35b76a18.png
1: stopwatch (0.98)
2: digital_watch (0.01)
3: analog_clock (0.00)
4: barometer (0.00)
5: scale (0.00)
Found 'watch' with probability 0.98
image_save_path_good good_watch_folder\02d816fba813ef3efc50fc522843c67b.png
1: matchstick (0.14)
2: fountain (0.13)
3: web_site (0.10)
4: cleaver (0.03)
5: hook (0.02)
image_save_path_bad bad_watch_folder\025211240c5c41717c3a8ab0fee71c24.png
1: stopwatch (0.96)
2: digital_watch (0.02)
3: analog_clock (0.01)
4: reflex_camera (0.00)
5: magnetic_compass (0.00)
Found 'watch' with probability 0.96
image_save_path_good good_w

...

Processing images:  99%|█████████▉| 1103/1116 [03:24<00:02,  5.47it/s]

1: stopwatch (0.85)
2: barometer (0.10)
3: analog_clock (0.04)
4: magnetic_compass (0.01)
5: digital_watch (0.00)
Found 'watch' with probability 0.85
image_save_path_good good_watch_folder\fe568a07e910431fb0883a3d6cd51f0c.png
1: magnetic_compass (0.20)
2: stopwatch (0.18)
3: reflex_camera (0.16)
4: analog_clock (0.09)
5: digital_watch (0.08)
Found 'watch' with probability 0.18
image_save_path_good good_watch_folder\fef75b52e17a25cad29a11411a846eee.png
1: stopwatch (1.00)
2: binoculars (0.00)
3: scale (0.00)
4: digital_watch (0.00)
5: analog_clock (0.00)
Found 'watch' with probability 1.00
image_save_path_good good_watch_folder\ff63e6d292ad6b23be4c83673e4db47b.png
1: stopwatch (0.92)
2: analog_clock (0.07)
3: digital_watch (0.01)
4: reflex_camera (0.00)
5: magnetic_compass (0.00)
Found 'watch' with probability 0.92
image_save_path_good good_watch_folder\fe43f2a40fe07997908bbe4c81b07522.png
1: stopwatch (0.97)
2: barometer (0.01)
3: analog_clock (0.01)
4: magnetic_compass (0.01)
5: digit

Processing images:  99%|█████████▉| 1107/1116 [03:25<00:01,  7.72it/s]

1: stopwatch (0.98)
2: analog_clock (0.02)
3: digital_watch (0.00)
4: barometer (0.00)
5: wall_clock (0.00)
Found 'watch' with probability 0.98
image_save_path_good good_watch_folder\fedc7824684b90c21f216e42e8ad64b5.png
1: stopwatch (0.99)
2: magnetic_compass (0.00)
3: barometer (0.00)
4: digital_watch (0.00)
5: analog_clock (0.00)
Found 'watch' with probability 0.99
image_save_path_good good_watch_folder\ffc3df0ca77aee2c1019c1f8759b4ad7.png
1: stopwatch (0.95)
2: magnetic_compass (0.01)
3: analog_clock (0.01)
4: barometer (0.01)
5: digital_watch (0.01)
Found 'watch' with probability 0.95
image_save_path_good good_watch_folder\ff6e32cd0e51701ac1c4632250f757aa.png


Processing images: 100%|██████████| 1116/1116 [03:25<00:00,  5.42it/s]


In [8]:
# Of the white background images, how many are good watch images

good_images_folder = Path("good_watch_folder")
num_items = len(list(good_images_folder.iterdir()))

print(f"Number of items in '{good_images_folder}': {num_items}")

Number of items in 'good_watch_folder': 1006


In [9]:
# Of the white background images, how many are bad watch images

bad_images_folder = Path("bad_watch_folder")
num_items = len(list(bad_images_folder.iterdir()))

print(f"Number of items in '{bad_images_folder}': {num_items}")

Number of items in 'bad_watch_folder': 107


### Combine transparent images + good white background into  good_images_PLUS_transparent folder

In [10]:
final_folder = "good_images_plus_transparent"
transparent_images = Path("transparent_images")
new_path = Path(final_folder)
        
if os.path.exists(new_path):
    try:
        shutil.rmtree(new_path)
        print(f"Directory '{new_path}' and all its contents have been deleted.")
    except Exception as e:
        print(f"Error: Failed to delete '{new_path}'. {e}")
  
if not os.path.exists(new_path):
    try:        
        shutil.copytree(good_images_folder, new_path)
        for file in transparent_images.iterdir():
            if file.is_file():
                destination_file = new_path / file.name
                shutil.copy2(file, destination_file)
        print(f"{final_folder} now contains good and transparent images.")
        
    except Exception as e:
        print(f"An error occurred: {e}")



Directory 'good_images_plus_transparent' and all its contents have been deleted.
good_images_plus_transparent now contains good and transparent images.


In [11]:
# Of the white background images, how many are good watch images PLUS original transparent images

final_folder = Path("good_images_PLUS_transparent")
num_items = len(list(final_folder.iterdir()))

print(f"Number of items in '{final_folder}': {num_items}")

Number of items in 'good_images_PLUS_transparent': 1347
